In [ ]:
# Change directory to the root so that relative path loads work correctly
import os

try:
    os.chdir(os.path.join(os.getcwd(), ".."))
    print(os.getcwd())
except:
    pass

In [ ]:
import os
import sys

import torch

from experiments.A_proof_of_constraint.visualize import (
    plot_constraints,
    plot_loss,
    plot_time,
)

In [ ]:
# Files to load
directory = "/global/u1/g/gelijerg/Projects/pyinsulate/results"
files = [
    # unconstrained
    "proof-of-constraint_2019-07-25-09-01-01.pth",
    # average
    "proof-of-constraint_2019-07-25-09-31-56.pth",
    # batchwise
    "proof-of-constraint_2019-07-25-09-40-10.pth",
]

In [ ]:
# Load files
summaries = [torch.load(f"{directory}/{f}") for f in files]
filenames = [os.path.splitext(f)[0] for f in files]

In [ ]:
# Do some plotting
for summary, filename in zip(summaries, filenames):
    training_monitor = summary["training_monitor"]
    evaluation_train_monitor = summary["evaluation_train_monitor"]
    evaluation_test_monitor = summary["evaluation_test_monitor"]

    print(summary["configuration"])
    #     print(training_monitor)
    #     print(evaluation_test_monitor.mean_loss)
    plot_loss(
        [evaluation_train_monitor, evaluation_test_monitor],
        ["Training", "Testing"],
        f"training-loss_{filename}",
    )
    plot_loss(
        [evaluation_train_monitor, evaluation_test_monitor],
        ["Training", "Testing"],
        f"training-constrained-loss_{filename}",
        title="Constrained losses",
        constrained=True,
    )
    plot_constraints(
        [evaluation_train_monitor, evaluation_test_monitor],
        ["Training", "Testing"],
        f"training-constraint_{filename}",
    )

    plot_time(training_monitor, f"compute-time_{filename}")

In [ ]:
print("done!")